<a href="https://colab.research.google.com/github/alex-smith-uwec/NLP_Spring2025/blob/main/Basic%20Text%20Normalization%20and%20Counting-Part%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Import corpus from Project Gutenberg

Bigram analysis


# Import a corpus from [Project Gutenberg](https://www.gutenberg.org/)

In [1]:
import requests

# Fetch book (Plain Text UTF-8)
url = "https://www.gutenberg.org/cache/epub/1342/pg1342.txt" #Jane Austen, Pride and Prejudice
response = requests.get(url)
text = response.text

print(text[:500])


﻿The Project Gutenberg eBook of Pride and Prejudice
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this


In [2]:
# Strip headers and footers
start_index = text.find("Chapter I.]")
end_index = text.find("*** END OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***")
corpus = text[start_index:end_index]

# Display first 500 characters
print(f"first words:\n {corpus[:500]}\n\n")

print(f"last words:\n\n\n {corpus[-400:]}")

first words:
 Chapter I.]


It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding families, that he is considered as the rightful
property of some one or other of their daughters.

“My dear Mr. Bennet,” said his lady to him one day, “have you heard that
Netherfield Park is l


last words:


 gratitude towards the persons who, by bringing
her into Derbyshire, had been the means of uniting them.

                            [Illustration:

                                  THE
                                  END
                                   ]




             CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.







In [3]:
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords


from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

from nltk import bigrams
from nltk.probability import ConditionalFreqDist

import random


In [5]:
# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#Tokenize the corpus into words
words = word_tokenize(corpus.lower())

# Remove punctuation
filtered_words = [word for word in words if word.isalnum()]
no_stop_filtered_words = [word for word in filtered_words if word not in stopwords.words('english')]

In [8]:
finder = BigramCollocationFinder.from_words(filtered_words)
finder_no_stops = BigramCollocationFinder.from_words(no_stop_filtered_words)

In [10]:
N=3 #filter to bigrams that appear at least N times
finder.apply_freq_filter(N)  # Only keep bigrams that occur at least N times
frequent_bigrams = finder.nbest(BigramAssocMeasures.raw_freq, 20)

frequent_bigrams

[('of', 'the'),
 ('to', 'be'),
 ('in', 'the'),
 ('i', 'am'),
 ('of', 'her'),
 ('it', 'was'),
 ('to', 'the'),
 ('of', 'his'),
 ('she', 'was'),
 ('she', 'had'),
 ('had', 'been'),
 ('it', 'is'),
 ('i', 'have'),
 ('that', 'he'),
 ('to', 'her'),
 ('could', 'not'),
 ('for', 'the'),
 ('he', 'had'),
 ('and', 'the'),
 ('he', 'was')]

In [13]:
N=3
finder_no_stops.apply_freq_filter(N)  # Only keep bigrams that occur at least N times
frequent_bigrams_no_stops = finder_no_stops.nbest(BigramAssocMeasures.raw_freq, 20)

frequent_bigrams_no_stops

[('lady', 'catherine'),
 ('miss', 'bingley'),
 ('miss', 'bennet'),
 ('said', 'elizabeth'),
 ('sir', 'william'),
 ('de', 'bourgh'),
 ('miss', 'darcy'),
 ('young', 'man'),
 ('1894', 'george'),
 ('colonel', 'fitzwilliam'),
 ('colonel', 'forster'),
 ('dare', 'say'),
 ('elizabeth', 'could'),
 ('young', 'ladies'),
 ('miss', 'lucas'),
 ('illustration', 'chapter'),
 ('cried', 'elizabeth'),
 ('said', 'bennet'),
 ('uncle', 'aunt'),
 ('great', 'deal')]

In [19]:
def generate_model(cfdist, word, num=15):
    for i in range(num):
        print(word, end=' ')
        # Get the most frequent next words (up to 3)
        next_words = list(cfdist[word].keys())[:3]
        if next_words:
            # Randomly choose from the top 3 words
            word = random.choice(next_words)
        else:
            break


# Generate bigrams from your list of words
filtered_bigrams = bigrams(filtered_words)

# Create a Conditional Frequency Distribution (CFD) from these bigrams
cfd = ConditionalFreqDist(filtered_bigrams)

# Example usage
generate_model(cfd, 'miss')  # Replace 'your_seed_word' with your desired starting word


miss lucas her life it is considered by the feelings towards him if you must 

In [20]:
from collections import Counter, defaultdict # Import Counter and defaultdict


tokens=filtered_words

# Count unigrams and bigrams
unigram_counts = Counter(tokens)
bigram_counts = Counter(bigrams(tokens))

# Calculate bigram probabilities with Laplace smoothing
vocab_size = len(unigram_counts)
bigram_probs = defaultdict(lambda: 1 / vocab_size)  # Default probability for unseen bigrams

for (w1, w2), count in bigram_counts.items():
    bigram_probs[(w1, w2)] = (count + 1) / (unigram_counts[w1] + vocab_size)

# Test the bigram model: Probability of "this is"
print(f"P('is' | 'this') = {bigram_probs[('this', 'is')]}")

# Generate text using the model


def generate_bigram_text(start_word, length=20):
    text = [start_word]
    for _ in range(length - 1):
        next_words = [pair[1] for pair in bigram_probs if pair[0] == text[-1]]
        if not next_words:  # No valid next word
            break
        next_word = random.choice(next_words)
        text.append(next_word)
    return ' '.join(text)

print("Generated Text:", generate_bigram_text('miss'))


P('is' | 'this') = 0.004526935264825713
Generated Text: miss lucas for were speaking out they came out if lady lives with their uncle if inconvenient and sunk into
